# Problema 3
$$
\begin{array}{ll}
\operatorname{minimizar} & \frac{1}{2} \vec{\boldsymbol{x}}^{\top} G \vec{\boldsymbol{x}}+\vec{\boldsymbol{c}}^{\top} \vec{\boldsymbol{x}} \\
\text { sujeto a } & A \vec{x}=\vec{\boldsymbol{b}} \\
& x_{i} \geq \ell_{i} \quad \text { si } \ell_{i} \text { es finito, } \\
& x_{i} \leq u_{i} \quad \text { si } u_{i} \text { es finito. }
\end{array}
$$

- Definimos $J \subset I$ como sigue:

- Para $x_{j} \geq \ell_{j}$ definimos $\left|g_{j}\left(x_{0}\right)\right| \leq 8 \varepsilon_{m} \max \left\{\left|\ell_{j}\right|, 1\right\} \Longrightarrow j \in J$

- Para $x_{j} \leq u_{j}$ definimos $\left|g_{j}\left(x_{0}\right)\right| \leq 8 \varepsilon_{m} \max \left\{\left|u_{j}\right|, 1\right\} \Longrightarrow j \in J$

Donde $\varepsilon_{m}$ es el épsilon de la máquina `eps(Float64)`

In [1]:
include("Solvers.jl")
using .Solvers, LinearAlgebra, MAT, .Solvers.Utils

In [15]:
problem = matread("../data/lp_afiro.mat")["Problem"]

Dict{String, Any} with 11 entries:
  "A"      => …
  "aux"    => Dict{String, Any}("c"=>[0.0; 0.0; … ; 0.0; 10.0], "hi"=>[Inf; Inf…
  "b"      => [0.0; 0.0; … ; 310.0; 300.0]
  "date"   => ""
  "name"   => "LPnetlib/lp_afiro"
  "kind"   => "linear programming problem"
  "author" => "M. Saunders"
  "id"     => 597.0
  "notes"  => ["A Netlib LP problem, in lp/data.  For more information", "send …
  "title"  => "Netlib LP problem afiro: minimize c'*x, where Ax=b, lo<=x<=hi"
  "ed"     => "D. Gay"

In [16]:
b = problem["b"] 
c = problem["aux"]["c"][:] # El [:] es para interpretar como vector
l = problem["aux"]["lo"][:] # El [:] es para interpretar como vector
u = problem["aux"]["hi"][:] # El [:] es para interpretar como vector
n_eq = length(b)
A_eq = problem["A"]
G = I(length(c))

# Como all(isfinite.(l)) == true, se complen todas las cotas inferiores
A = [A_eq; -I(length(l))]
b = [b; -l]

# Imponiendo cotas superiores
# Concatenando las filas I_j de una identidad tal que u_j < Inf
mask = isfinite.(u)
A = [A; I(length(u))[mask, :]]
# Igual cambiando b para que dimensiones coincidan
b = [b; u[mask]]
b = b[:]

78-element Vector{Float64}:
  0.0
  0.0
 80.0
  0.0
  0.0
  0.0
 80.0
  0.0
  0.0
  0.0
  ⋮
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0

In [17]:
# Definiendo el conjunto inicial de igualdades
# Obtenemos x_0 con linprog (pero ya sabemos que va a ser vec(0))
x_0 = linprog(A, b, n_eq)

# Shorthand para epsilon de maquina
εₘ = eps(Float64)

# Obteniendo los masks que indican si se usa la restricción
# Recordamos que para las cotas inferiores g_i(x) = l_i - x_i & para las sup g_i(x) = x_i - u_i
J_l = abs.(l - x_0) .<= 8 * εₘ *  max.(abs.(l), ones(length(l)))

# Ahora para las superiores, recordando quitar los renglones donde u_j no es finito (por size de A).
J_u = abs.(x_0 - u) .<= 8 * εₘ *  max.(abs.(u), ones(length(u)))
J_u = J_u[isfinite.(u)]

J = [J_l; J_u]

# Concatenando con un índice aleatorio
R_index = rand(findall(J), 1)
notJ = falses(size(A, 1) - n_eq)
notJ[R_index] .= true
W_0 = [trues(n_eq); notJ]

78-element BitVector:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [18]:
println(W_0)

Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Nótese que el número de filas de W_0 coincide con el de A, y el tamaño de b

In [19]:
@assert size(A, 1) == length(W_0)
@assert size(A, 1) == length(b)

In [20]:
iters, x_star, q_star, μ = activeSetMethod(G, c, A, b, n_eq, copy(W_0))

α = -0.0, j = 29 

Rama 1. ||d_k|| = 270.8, q(x) = 323100.0, α=-0.0k = 0
α = -0.0, j = 44 

Rama 1. ||d_k|| = 270.9, q(x) = 323100.0, α=-0.0k = 1
α = -0.0, j = 69 

Rama 1. ||d_k|| = 268.1, q(x) = 323100.0, α=-0.0k = 2
α = -0.0, j = 68 

Rama 1. ||d_k|| = 267.9, q(x) = 323100.0, α=-0.0k = 3
α = -0.0, j = 70 

Rama 1. ||d_k|| = 267.9, q(x) = 323100.0, α=-0.0k = 4
α = -0.0, j = 71 

Rama 1. ||d_k|| = 267.4, q(x) = 323100.0, α=-0.0k = 5
α = -0.0, j = 35 

Rama 1. ||d_k|| = 214.3, q(x) = 323100.0, α=-0.0k = 6
α = -0.0, j = 55 

Rama 1. ||d_k|| = 215.4, q(x) = 323100.0, α=-0.0k = 7
α = -0.0, j = 31 

Rama 1. ||d_k|| = 215.7, q(x) = 323100.0, α=-0.0k = 8
α = -0.0, j = 54 

Rama 1. ||d_k|| = 215.7, q(x) = 323100.0, α=-0.0k = 9
α = -0.0, j = 78 

Rama 1. ||d_k|| = 215.7, q(x) = 323100.0, α=-0.0k = 10
α = -0.0, j = 53 

Rama 1. ||d_k|| = 232.2, q(x) = 323100.0, α=-0.0k = 11
α = 1.206, j = 28 

Rama 1. ||d_k|| = 232.2, q(x) = 201000.0, α=1.206
Rama 2. 
α = 3.033, j = 41 

Rama 1. ||d_k|| = 13.14

51-element Vector{Float64}:
  15.09
   1.688e-14
  33.95
   1.332e-15
   2.336
   2.357
 264.5
  -9.948e-14
 358.4
   2.153
   ⋮
  -8.527e-14
   5.684e-14
   2.153
   2.182
   2.211
 113.5
  15.94
  60.88
  -2.274e-13

Concluyó método del conjunto activo en 14 iteraciones
El punto de paro fue:


In [21]:
print(round.(x_star, sigdigits=4))

[15.09, 1.688e-14, 33.95, 1.332e-15, 2.336, 2.357, 264.5, -9.948e-14, 358.4, 2.153, 2.182, 2.211, 1.94, 1.928, 10.62, 13.14, -5.684e-14, 139.9, 183.3, 64.91, 37.31, 27.6, 68.8, 46.05, 6.65, 3.042e-14, 7.105e-15, -1.421e-14, 6.65, 2.336, 2.357, 26.65, 26.05, 55.87, 235.5, 158.9, 32.68, 43.88, 101.3, 141.6, -8.527e-14, -1.137e-13, -8.527e-14, 5.684e-14, 2.153, 2.182, 2.211, 113.5, 15.94, 60.88, -2.274e-13]

In [22]:
q_star

401820.55566481256

## Benchmarks

In [26]:
using BenchmarkTools
using Plots
pgfplotsx(); theme(:ggplot2)

# Corriendo el benchmark de acuerdo a las best practices
bench = @benchmark activeSetMethod($G, $c, $A, $b, $n_eq, W_k) setup=(W_k = copy(W_0));

In [27]:
bench

BenchmarkTools.Trial: 630 samples with 1 evaluation.
 Range (min … max):  5.764 ms … 29.671 ms  ┊ GC (min … max): 0.00% … 52.42%
 Time  (median):     7.294 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.912 ms ±  3.213 ms  ┊ GC (mean ± σ):  5.72% ± 10.52%

  █▆▁    ▂▁                                                   
  ███▆▅▆▇██▆▄▄▃▂▃▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂ ▃
  5.76 ms        Histogram: frequency by time          25 ms <

 Memory estimate: 2.34 MiB, allocs estimate: 8800.

In [28]:
ts = bench.times
h = histogram(
    ts,
    xlabel="Tiempo (s)",
    legend = :none,
    framestyle = :box);

In [29]:
savefig(h, "../docs/histograma2.tex")